# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [32]:
# import libraries
import joblib
import pandas as pd
import re
import pickle

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sqlalchemy import create_engine
from time import time
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('df',engine)
X = df['message']
Y = df.iloc[:,4:]

In [4]:
df.head()

,id,message,original,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,direct,news
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,1,0,0,1,0,0,0,...,1,0,1,0,0,0,0,0,1,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
def model_pipeline():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier())),
    ])

    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

model = []
model = model_pipeline()
model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
msg = ['Hello I see fire in the street and many houses are destroyed, homeless people everywhere']
test_output = model.predict(msg)
print(y_train.columns.values[(test_output.flatten()==1)])

['request' 'aid_related' 'weather_related' 'direct']


In [12]:
y_pred = model.predict(X_test)

In [18]:
overall_accuracy = (y_pred == y_test).mean().mean()

print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))

Average overall accuracy 94.53% 



In [19]:
y_pred_pd = pd.DataFrame(y_pred, columns = y_test.columns)
for column in y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(y_test[column],y_pred_pd[column]))

------------------------------------------------------

FEATURE: request

             precision    recall  f1-score   support

          0       0.89      0.99      0.93      5438
          1       0.86      0.38      0.52      1116

avg / total       0.88      0.88      0.86      6554

------------------------------------------------------

FEATURE: offer

             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6520
          1       0.00      0.00      0.00        34

avg / total       0.99      0.99      0.99      6554

------------------------------------------------------

FEATURE: aid_related

             precision    recall  f1-score   support

          0       0.71      0.88      0.79      3788
          1       0.76      0.51      0.61      2766

avg / total       0.73      0.73      0.71      6554

------------------------------------------------------

FEATURE: medical_help

             precision    recall  f1-score   support

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [26]:
def new_model_pipeline():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(OneVsRestClassifier(LogisticRegression(random_state = 42)), n_jobs=-1))
    ])

    return pipeline

In [27]:
model = new_model_pipeline()

parameters = {
    'clf__estimator__estimator__C': [1, 2, 5],
    'clf__estimator__estimator__penalty': ['l1', 'l2']}

cv = GridSearchCV(model, param_grid=parameters, scoring = 'precision_samples',cv=5)

cv.fit(X_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWa

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...r', tol=0.0001,
          verbose=0, warm_start=False),
          n_jobs=1),
           n_jobs=-1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__estimator__C': [1, 2, 5], 'clf__estimator__estimator__penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='precision_samples', verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [28]:
y_pred_new = cv.predict(X_test)

In [29]:
overall_accuracy_new = (y_pred_new == y_test).mean().mean()

print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy_new*100))

Average overall accuracy 95.31% 



In [30]:
y_pred_pd_new = pd.DataFrame(y_pred_new, columns = y_test.columns)
for column in y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(y_test[column],y_pred_pd_new[column]))

------------------------------------------------------

FEATURE: request

             precision    recall  f1-score   support

          0       0.92      0.97      0.94      5438
          1       0.81      0.58      0.67      1116

avg / total       0.90      0.90      0.90      6554

------------------------------------------------------

FEATURE: offer

             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6520
          1       0.00      0.00      0.00        34

avg / total       0.99      0.99      0.99      6554

------------------------------------------------------

FEATURE: aid_related

             precision    recall  f1-score   support

          0       0.79      0.86      0.82      3788
          1       0.78      0.69      0.73      2766

avg / total       0.79      0.79      0.79      6554

------------------------------------------------------

FEATURE: medical_help

             precision    recall  f1-score   support

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

After using gridSearch accuracy improved from 94.53 to 95.31

### 9. Export your model as a pickle file

In [33]:
# save the model to disk
filename = 'classifier.sav'
pickle.dump(model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.